# Init & Login

In [1]:
from math import ceil
from random import randrange
from time import sleep

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = None
token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

print( token )

spot = spotipy.Spotify( auth = token )
clear_output( wait = True )
sleep( 2 )
print( "TOKEN OBTAINED" )

TOKEN OBTAINED


# Playlist Functions

In [2]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']


def reorder_playlist_by_chunks( playlist_ID, chunkSize, Nmoves = None, verbose = False ):
    """ Shuffle the playlist a chunk at a time """
    Ntracks = get_playlist_length( playlist_ID )
    Nchunks = int( ceil( Ntracks / chunkSize ) )
    if Nmoves is None:
        Nmoves = int( Ntracks / chunkSize )
    for i in range( Nmoves ):
        bgnDex   = randrange(0,Nchunks) * chunkSize
        moveSize = min( chunkSize, Ntracks - bgnDex - 1 )
        sendDex  = randrange(0,Ntracks-1)
        try:
            response = spot.playlist_reorder_items(
                playlist_ID, 
                bgnDex, 
                sendDex, 
                range_length = chunkSize
            )
            if verbose:
                print( f"Iteration {i+1}, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{response}" )
        except spotipy.SpotifyException as ex:
            if verbose:
                print( f"ERROR, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{ex}" )


def complete_reorder_entire_playlist( playlist_ID, verbose = False, phasePause_s = 5 ):
    """ Completely shuffle the playlist in decreasing chunks """
    Ntracks = get_playlist_length( playlist_ID )
    ## Phase 1: Chunks of 100 ##
    if verbose:
        print("\nPhase 1: Chunks of 100\n")
    reorder_playlist_by_chunks( playlist_ID, 100, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 2: Chunks of 10 ##
    if verbose:
        print("\nPhase 2: Chunks of 10\n")
    reorder_playlist_by_chunks( playlist_ID, 10, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: Chunks of 5, 1/2 ##
    if verbose:
        print("\nPhase 3: Chunks of 5, 1/2\n")
    Nmoves = int( ceil( Ntracks / 10 ) )
    reorder_playlist_by_chunks( playlist_ID, 5, Nmoves, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: 1 at a Time, 1/3 ##
    if verbose:
        print("\nPhase 3: 1 at a Time, 1/3\n")
    Nmoves = int( ceil( Ntracks / 3 ) )
    reorder_playlist_by_chunks( playlist_ID, 1, Nmoves, verbose = verbose )

# Shuffle Playlists!

In [3]:
playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'study12' : "0ecSAfnD4CulIVnLt26ukI",
    'study13' : "7K9ucByFRgDuZk8KMHeJkL",
    'study14' : "0v26bHydUxcGC5EbMlkjzG",
    'study15' : "6SqlfurCBP7eeMOojaDNtS",
    'cringe'  : "2AAUYlKM1nXKHkZUpUSFbv",
}

complete_reorder_entire_playlist( playlist['study10'], verbose = True, phasePause_s = 5 )


Phase 1: Chunks of 100



HTTP Error for PUT to https://api.spotify.com/v1/playlists/3832xeKGEOAXFJqE4K8kIq/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 1, 100-200 --> 114:
	{'snapshot_id': 'AAADq0ryegYiVrHcumT8scF9DBejpHZ0'}
Iteration 2, 200-300 --> 309:
	{'snapshot_id': 'AAADrMtvgB5a0ClKNGvrqGSD3/oBbSvq'}
ERROR, 300-398 --> 92:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/3832xeKGEOAXFJqE4K8kIq/tracks:
 Tracks selected to be reordered are out of bounds, reason: None

Phase 2: Chunks of 10



HTTP Error for PUT to https://api.spotify.com/v1/playlists/3832xeKGEOAXFJqE4K8kIq/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


ERROR, 390-398 --> 296:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/3832xeKGEOAXFJqE4K8kIq/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 2, 330-340 --> 84:
	{'snapshot_id': 'AAADrbwLrioKsYoak+sue/Ni4hASoIvh'}
Iteration 3, 140-150 --> 143:
	{'snapshot_id': 'AAADrjPIE61iFSNku/9s21+hOjABYWIP'}
Iteration 4, 330-340 --> 333:
	{'snapshot_id': 'AAADr4y/avEcZUwzON1TmnOXqNY0Kpq2'}
Iteration 5, 190-200 --> 264:
	{'snapshot_id': 'AAADsEvL8VkMznw79bvq4pLS2kKwgcX1'}
Iteration 6, 60-70 --> 278:
	{'snapshot_id': 'AAADseJY4HzmpiKgH/IaDV/EqYEMRlzg'}
Iteration 7, 80-90 --> 111:
	{'snapshot_id': 'AAADsv/7chvFtVA14thYucRkNbd8AcSC'}
Iteration 8, 110-120 --> 2:
	{'snapshot_id': 'AAADs6do538tg1JcnxwpwYI0Z5bVnkNK'}
Iteration 9, 180-190 --> 265:
	{'snapshot_id': 'AAADtDY4FzFhy+LB0W/JST0F59KHYpcY'}
Iteration 10, 200-210 --> 99:
	{'snapshot_id': 'AAADtT9ctqLxUBvhAukl8ACnSJ591HXr'}
Iteration 11, 290-300 --> 263:
	{'snapshot_id': 'AAADttHb5eZ5awXluoYlkxFrM

...

Iteration 16, 330-340 --> 336:
	{'snapshot_id': 'AAADu1g3UaFArsalpmhZ6hVouupFfcYg'}
Iteration 17, 160-170 --> 156:
	{'snapshot_id': 'AAADvLeg0MKRG0DToBIL02jHXsVPW9xR'}
ERROR, 390-398 --> 364:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/3832xeKGEOAXFJqE4K8kIq/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 19, 40-50 --> 344:
	{'snapshot_id': 'AAADvaFB7T2ErZTv92OpSI0fcr0A9GgM'}
Iteration 20, 240-250 --> 120:
	{'snapshot_id': 'AAADvqd0jxvzbk8jOjib9iWMR5qxlAQx'}
Iteration 21, 140-150 --> 29:
	{'snapshot_id': 'AAADv2ZTNLmhd+lGL8qck5EyXiNCR8D0'}
Iteration 22, 340-350 --> 46:
	{'snapshot_id': 'AAADwJCuXuaRNquFM4FlF+elRhQHmLRp'}
Iteration 23, 120-130 --> 310:
	{'snapshot_id': 'AAADwT1h13Ew3iygb3wAFdORg6vTRHao'}
Iteration 24, 310-320 --> 92:
	{'snapshot_id': 'AAADwgryIA+YRAX59u2jzyXvblFuTG8Y'}
Iteration 25, 30-40 --> 384:
	{'snapshot_id': 'AAADww83SfT2KNqEY6VtvogD7BZME23C'}
Iteration 26, 110-120 --> 381:
	{'snapshot_id': 'AAADxPeCpkHABD2W

HTTP Error for PUT to https://api.spotify.com/v1/playlists/3832xeKGEOAXFJqE4K8kIq/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 21, 315-320 --> 68:
	{'snapshot_id': 'AAAD5ggmaAbjeISHeJUl1khgxfL6Hbe+'}
Iteration 22, 100-105 --> 212:
	{'snapshot_id': 'AAAD52xbvaIT1eo1y5IHlbsPkwx4Z4nX'}
ERROR, 395-398 --> 276:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/3832xeKGEOAXFJqE4K8kIq/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 24, 280-285 --> 387:
	{'snapshot_id': 'AAAD6GBkxqesz+Xs+YUJdM+RPstr0XSS'}
Iteration 25, 295-300 --> 57:
	{'snapshot_id': 'AAAD6cRgmCDWR/YptsM6mmYXXBityCxk'}
Iteration 26, 135-140 --> 17:
	{'snapshot_id': 'AAAD6m+TCRgHTseN4m8I3hxnvOdSWfAO'}
Iteration 27, 125-130 --> 374:
	{'snapshot_id': 'AAAD65c5Wh3Ekqv+utPKC7Qzt9i/HrVB'}
Iteration 28, 385-390 --> 20:
	{'snapshot_id': 'AAAD7Ou8qjYO2z2QyelLP4c+h5NLABRL'}
Iteration 29, 135-140 --> 393:
	{'snapshot_id': 'AAAD7aqEgdC5BKXzhDMAI3WNQx8Yuwso'}
Iteration 30, 30-35 --> 184:
	{'snapshot_id': 'AAAD7gA54S41sJFKod0ei0SQzcWOK0Xs'}
Iteration 31, 55-60 --> 206:
	{'snapshot_id': 'AAAD76XqPhokooZsd

HTTP Error for PUT to https://api.spotify.com/v1/playlists/3832xeKGEOAXFJqE4K8kIq/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


ERROR, 395-398 --> 162:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/3832xeKGEOAXFJqE4K8kIq/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 33, 270-275 --> 87:
	{'snapshot_id': 'AAAD8KpVhErulD6n5wn6nOEEi/WlpLtp'}
Iteration 34, 195-200 --> 318:
	{'snapshot_id': 'AAAD8SifiHviduAtneVBxjppUNzMz0ZF'}
Iteration 35, 320-325 --> 120:
	{'snapshot_id': 'AAAD8sFChbqrpWNtxNw+EUCBbVJbDsRs'}
Iteration 36, 315-320 --> 277:
	{'snapshot_id': 'AAAD8wSMwyAIVzxV5hJIrd65RVMPy6RP'}
Iteration 37, 205-210 --> 151:
	{'snapshot_id': 'AAAD9CAYF7fdnOnwZru7Fj4/4GQbyipP'}
Iteration 38, 250-255 --> 74:
	{'snapshot_id': 'AAAD9QwscS0asbQHLrHw2jvMJmI0JdLK'}
Iteration 39, 130-135 --> 171:
	{'snapshot_id': 'AAAD9vgneKiLHwr4eXNki4s/kJmVmVzU'}
Iteration 40, 50-55 --> 116:
	{'snapshot_id': 'AAAD95h7nVSEZiU1BcVXJRr/+AtlQ9Cb'}

Phase 3: 1 at a Time, 1/3

Iteration 1, 79-80 --> 41:
	{'snapshot_id': 'AAAD+Nrj9U9n0c8efUr3ni1n4h3LZHPZ'}
Iteration 2, 206-207 --> 305:
	{'snaps